In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append('../Augur')
from transformers import (AutoConfig, 
                          AutoModelForCausalLM, 
                          AutoTokenizer, 
                          TextIteratorStreamer, 
                          BitsAndBytesConfig, 
                          TextStreamer,
                          Conversation, 
                          pipeline,
                          LlamaForCausalLM)
import torch

from Augur import model

In [2]:
def load_quant_model(model_id):
    # This requires the `bitsandbytes` library
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model_config = AutoConfig.from_pretrained(model_id)

    _model = model.AugurLlamaModel.from_pretrained(
        model_id,
        trust_remote_code=True,
        config=model_config,
        quantization_config=bnb_config,
        device_map='auto',
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    return _model, tokenizer

In [3]:
models, tokenizer = load_quant_model('codellama/CodeLlama-13b-Instruct-hf')

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
output = models.generate(do_sample=True, num_beams=3, num_beam_groups=1, num_return_sequences=3, diversity_penalty=0.0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [39]:
pipe= pipeline('text-generation', model=models, tokenizer=tokenizer)

The model 'AugurLlamaModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCau

In [35]:
conver = Conversation()
conver.add_user_input('Once upon a time')
streamer = TextStreamer(tokenizer=tokenizer)

In [76]:
good_shit = tokenizer.decode(tokenizer.apply_chat_template([{'role':'user', 'content':'generate the dbpedia sparql code that answer the following question: retrieve all the authors of the book harry potter'}]))

In [96]:
#pipe_output = pipe(good_shit, max_length=250, do_sample= True, num_beams=4, num_return_sequences=4, diversity_penalty=0.0)
pipe_output = [pipe(good_shit, max_length=250, do_sample=True) for _ in range(4)]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [97]:
pipe_output

[[{'generated_text': '<s> [INST] generate the dbpedia sparql code that answer the following question: retrieve all the authors of the book harry potter [/INST]  Here is an example of a DBpedia SPARQL query that retrieves the authors of the book "Harry Potter" using the prefixes and patterns found in the [DBpedia dataset](https://wiki.dbpedia.org/):\n\n```\nPREFIX dbpedia: <http://dbpedia.org/resource/>                       \nPREFIX dbpedia2: <http://dbpedia.org/data/>                          \nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>                  \nPREFIX dcterms: <http://purl.org/dc/terms/>                          \n\nSELECT?author\nWHERE\n{  \n ?book dbpedia:wikiPageRedirects?book_id.\n ?book dbpedia:wikiPageDisambiguates?book_disamb.\n ?book_disamb dcterms:subject "Harry'}],
 [{'generated_text': '<s> [INST] generate the dbpedia sparql code that answer the following question: retrieve all the authors of the book harry potter [/INST]  Here is a DBpedia SPARQL query 

In [133]:
import re
def capture_code(text): 
    code_pattern = r"```(?:sparql)?(.*?)```"
    code = re.findall(code_pattern, text, re.DOTALL)
    return code 

In [135]:
print(pipe_output[0][0]['generated_text'])

<s> [INST] generate the dbpedia sparql code that answer the following question: retrieve all the authors of the book harry potter [/INST]  Here is an example of a DBpedia SPARQL query that retrieves the authors of the book "Harry Potter" using the prefixes and patterns found in the [DBpedia dataset](https://wiki.dbpedia.org/):

```
PREFIX dbpedia: <http://dbpedia.org/resource/>                       
PREFIX dbpedia2: <http://dbpedia.org/data/>                          
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>                  
PREFIX dcterms: <http://purl.org/dc/terms/>                          

SELECT?author
WHERE
{  
 ?book dbpedia:wikiPageRedirects?book_id.
 ?book dbpedia:wikiPageDisambiguates?book_disamb.
 ?book_disamb dcterms:subject "Harry


In [134]:
[print(capture_code(x[0]['generated_text'])) for x in pipe_output]

[]
['\nPREFIX dbpedia: <http://dbpedia.org/ontology/>\n\nSELECT DISTINCT?author\nWHERE {\n ?book dbpedia:abstract "Harry Potter"@en.\n ?book dbpedia:author?author.\n ?author a dbpedia:Person.\n ?author rdfs:label?authorLabel.\n}\n']
['\nPREFIX dbp-ont: <http://dbpedia.org/ontology/>\nPREFIX dcterms: <http://purl.org/dc/terms/>\n\nSELECT?author?authorName\nFROM <http://dbpedia.org/>\nWHERE {\n ?book dcterms:title "Harry Potter".\n ?author dcterms:creator?book.\n ?author foaf:name?authorName.\n}\n']
['\nprefix dbpedia: <http://dbpedia.org/resource/>\nprefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nprefix dbpprop: <http://dbpedia.org/property/>\n\nSELECT?author\nWHERE {\n ?book dbpedia:name "Harry Potter"@en.\n ?book dbpprop:author?author.\n}\n']


[None, None, None, None]

In [30]:
output

tensor([[    1, 29871, 30143,  4746,  2184, 29936,    13,  4746,  2184, 29889,
         19466, 29889, 15809, 29936,    13,  4746,  2184, 29889, 11667, 29939],
        [    1, 29871, 30143,  4746,  2184, 29936,    13,  4746,  2184, 29889,
         19466, 29889, 15809, 29936,    13,  4746,  2184, 29889,  1626, 29936],
        [    1,    13,    13,    13,    13,    13,    13,    13,    13,    13,
            13,    13,    13,    13,    13,    13,    13,    13,    13,    13]],
       device='cuda:0')

In [22]:
caca = [tokenizer.decode(x) for x in output]

In [23]:
caca

['<s> \ufeffusing System;\nusing System.Collections.Generic;\nusing System.Linq',
 '<s> \ufeffusing System;\nusing System.Collections.Generic;\nusing System.Text;',
 '<s>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n']